# Belief propagation for quantum LDPC codes

Belief propagation generally works on factor graphs. In our context, we have a set of nodes that correspond to qubits, $V$, and another set of nodes that correspond to checks (factors in factor graph terminoplogy) $C$.

We will also have a set of edges $E$, that will connect nodes in $V$ to checks in $C$ (but not checks to checks or qubit nodes to qubit nodes). The set of edges, in our context, is given either by a parity check matrix, or a set of stabilizer generators.
When two checks $c,c'$ are both connected to $q,q'$, we will have a 4-cycle: $(q,c), (c,q'), (q',c'), (c',q)$.  


Plan for this workbook:

1.	Run bp+osd with a single code and parameters from the Degenerate Quantum LDPC Codes With Good Finite Length
Performance. If possible, add a code from Improved belief propagation is sufficient for real-time decoding of quantum memory (which is taken from here https://link.aps.org/accepted/10.1103/PhysRevA.88.012311), and same for a single code from Decoding Across the Quantum LDPC Code Landscape
2.	Set the number of iterations to 0, 1, 10, 20, 100, 1000, 10000 and plot graphs. Is there a difference ?
3.	So reading into 2005.07016, the conclusion is that you run X type error solely, then maybe if you want you can simulate Z type errors solely, but in any case you restrict yourself to hx or hz but not both.
4.	Start with the codes in here Degenerate Quantum LDPC Codes With Good Finite Length Performance
a.	A1
b.	B1 (if possible)
c.	C1 (if possible)
d.	Maybe D1 if possible


References:
1. Degenerate Quantum LDPC Codes With Good Finite Length Performance https://arxiv.org/pdf/1904.02703
2. Decoding Across the Quantum LDPC Code Landscape https://arxiv.org/pdf/2005.07016
3. Quantum Kronecker sum-product low-density parity-check codes with finite rate https://link.aps.org/accepted/10.1103/PhysRevA.88.012311
4. BP+OSD code https://github.com/quantumgizmos/bp_osd
5. High-threshold and low-overhead fault-tolerant quantum memory
6. On the iterative decoding of sparse quantum codes


Follow up that is not covered in this notebook yet:
1. Decoding Quantum Tanner 
2. Loopy Belief Propagation: Convergence and Effects of Message Errors
3. Probabilistic Graphical Models: A Concise Tutorial https://arxiv.org/pdf/2507.17116
4. Improved belief propagation is sufficient for real-time decoding of quantum memory https://arxiv.org/pdf/2506.01779



In [ ]:
import numpy as np
from ldpc import BpOsdDecoder
from ldpc.codes import rep_code
from bposd.hgp import hgp
from polynomialCodes import A1_HX, A1_HZ, A2_HX, A2_HZ, A3_HX, A3_HZ, A4_HX, A4_HZ, A5_HX, A5_HZ,  A6_HX, A6_HZ
from bposd.css import css_code
from scipy.linalg import lu
import matplotlib.pyplot as plt
import numpy as np
seed = 7134066
localRandom = np.random.RandomState(seed)

A1 = css_code(hx=A1_HX, hz=A1_HZ)
code = A1

#rangeFromDecodingAcrossTheQuantumLDPCCodeLandscape = [0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1]

#rangeFromDecodingAcrossTheQuantumLDPCCodeLandscape = rangeFromDecodingAcrossTheQuantumLDPCCodeLandscape
#rangeFromImprovedBeliefPropagationIsSufficientForRealTimeDecodingOfQuantumMemory = [0.001,0.002, 0.003, 0.004,0.005,0.006,0.007,0.008,0.009] 
#rangeFromImprovedBeliefPropagationIsSufficientForRealTimeDecodingOfQuantumMemory = rangeFromImprovedBeliefPropagationIsSufficientForRealTimeDecodingOfQuantumMemory
totalRange = np.linspace(0.0003, 0.1, 50)#rangeFromImprovedBeliefPropagationIsSufficientForRealTimeDecodingOfQuantumMemory + rangeFromDecodingAcrossTheQuantumLDPCCodeLandscape 
totalRange = [float(i) for i in totalRange]

numberOfIterations = [0,1,10,100,1000,10000]#,100,1000]

marker = 0
numberOfShots=100
berDictionary = {}
for iters in range(len(numberOfIterations)):
    berDictionary[f"Z_decode_ber_{numberOfIterations[iters]}_iterations"] = np.zeros(len(totalRange))
    berDictionary[f"X_decode_ber_{numberOfIterations[iters]}_iterations"] = np.zeros(len(totalRange))
    phyErrors = np.zeros(len(totalRange))
    k = 0
    for p_error in totalRange:
        bpdZ=BpOsdDecoder(A1.hz,#the parity check matrix
        error_rate=p_error,
        channel_probs=[None], #assign error_rate to each qubit. This will override "error_rate" input variable
        max_iter=numberOfIterations[iters], #the maximum number of iterations for BP)
        bp_method="ms",
        ms_scaling_factor=0, #min sum scaling factor. If set to zero the variable scaling factor method is used
        osd_method="osd0", #the OSD method. Choose from:  1) "osd_e", "osd_cs", "osd0"
        osd_order=0 #the osd search depth
        )   
        bpdX=BpOsdDecoder(A1.hx,#the parity check matrix
        error_rate=p_error,
        channel_probs=[None], #assign error_rate to each qubit. This will override "error_rate" input variable
        max_iter=numberOfIterations[iters], #the maximum number of iterations for BP)
        bp_method="ms",
        ms_scaling_factor=0, #min sum scaling factor. If set to zero the variable scaling factor method is used
        osd_method="osd0", #the OSD method. Choose from:  1) "osd_e", "osd_cs", "osd0"
        osd_order=0 #the osd search depth
        )
        print(f"Error Rate: {p_error}")
        xLogicalErrorCount = 0
        zLogicalErrorCount = 0
        for i in range(numberOfShots):
            error = localRandom.choice([0,1], size=code.N, replace=True, p=[1 - p_error, p_error])
            phyErrors[k]+=np.sum(error)
            syndrome=A1.hx@error %2
            bpdZ.decode(syndrome)
            residual_error = (bpdZ.osdw_decoding+error) % 2
            #Decoding is successful if the residual error commutes with the logical operators
            if (A1.lz@residual_error%2).any():
                zLogicalErrorCount+=1
            bpdX.decode(syndrome)
            residual_error = (bpdX.osdw_decoding+error) % 2
            #Decoding is successful if the residual error commutes with the logical operators
            if (A1.lx@residual_error%2).any():
                xLogicalErrorCount+=1
        print(f"Logical Errors: {xLogicalErrorCount}\n")
        print(f"Logical Errors: {zLogicalErrorCount}\n")
        berDictionary[f"X_decode_ber_{numberOfIterations[iters]}_iterations"][k]=xLogicalErrorCount
        berDictionary[f"Z_decode_ber_{numberOfIterations[iters]}_iterations"][k]=zLogicalErrorCount
        k +=1
    marker = marker +1
    


markers = ['-', '-o', '-^', '-+', '-*', '-x', '-s', '-d']
fig1, ax1 = plt.subplots(1, 1, layout='constrained')
ax1.set(title='A1 Code: BP+OSD Decoding Performance', xlabel='Physical Error Rate', ylabel='Rate of logical errors.')

for iters in range(len(numberOfIterations)):
    ax1.loglog(totalRange, berDictionary[f"ber_{numberOfIterations[iters]}_iterations"]/numberOfShots, markers[iters], label = f"BP+OSD Decoding for A1.hz with {numberOfIterations[iters]} BP iterations.")    
    #ax1.loglog(totalRange, phyErrors/(numberOfShots * code.N), '-', label = 'Physical Errors, actual (sanity check).')
# #ax1.axvspan(0, 0.1, facecolor='lightblue', alpha=0.3, 'Region tested by IBM')   # Left side background
# #ax1.axvspan(0.1, 0.5, facecolor='lightgreen', alpha=0.3, 'Region by Panteleev and Kalachev') # Right side background
ax1.legend()
ax1.grid()
ax1.grid(which="minor", color="0.9")



Error Rate: 0.0003
Logical Errors: 59

Logical Errors: 49

Error Rate: 0.0023346938775510207
Logical Errors: 36

Logical Errors: 60

Error Rate: 0.004369387755102041
Logical Errors: 29

Logical Errors: 84

Error Rate: 0.006404081632653063
Logical Errors: 12

Logical Errors: 97

Error Rate: 0.008438775510204083
Logical Errors: 8

Logical Errors: 97

Error Rate: 0.010473469387755103
Logical Errors: 6

Logical Errors: 99

Error Rate: 0.012508163265306125
Logical Errors: 5

Logical Errors: 99

Error Rate: 0.014542857142857146
Logical Errors: 2

Logical Errors: 99

Error Rate: 0.016577551020408168
Logical Errors: 1

Logical Errors: 100

Error Rate: 0.01861224489795919
Logical Errors: 1

Logical Errors: 100

Error Rate: 0.020646938775510208
Logical Errors: 0

Logical Errors: 100

Error Rate: 0.02268163265306123
Logical Errors: 0

Logical Errors: 100

Error Rate: 0.024716326530612252
Logical Errors: 0

Logical Errors: 100

Error Rate: 0.02675102040816327
Logical Errors: 0

Logical Errors: 100

In [ ]:
A1 = css_code(hx=A1_HX, hz=A1_HZ)
code = A1

#rangeFromDecodingAcrossTheQuantumLDPCCodeLandscape = [0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1]

#rangeFromDecodingAcrossTheQuantumLDPCCodeLandscape = rangeFromDecodingAcrossTheQuantumLDPCCodeLandscape
#rangeFromImprovedBeliefPropagationIsSufficientForRealTimeDecodingOfQuantumMemory = [0.001,0.002, 0.003, 0.004,0.005,0.006,0.007,0.008,0.009] 
#rangeFromImprovedBeliefPropagationIsSufficientForRealTimeDecodingOfQuantumMemory = rangeFromImprovedBeliefPropagationIsSufficientForRealTimeDecodingOfQuantumMemory
totalRange = np.linspace(0.0003, 0.01, 10)#rangeFromImprovedBeliefPropagationIsSufficientForRealTimeDecodingOfQuantumMemory + rangeFromDecodingAcrossTheQuantumLDPCCodeLandscape 
totalRange = [float(i) for i in totalRange]

for iters in range(len(numberOfIterations)):
    berDictionary[f"ber_{numberOfIterations[iters]}_iterations"] = np.zeros(len(totalRange))
    berArray = np.zeros(len(totalRange))
    k = 0
    for p_error in totalRange:
        bpd=bposd_decoder(A1.hx,#the parity check matrix
        error_rate=0.3,
        channel_probs=[None], #assign error_rate to each qubit. This will override "error_rate" input variable
        max_iter=100, #the maximum number of iterations for BP)
        bp_method="ms",
        ms_scaling_factor=0, #min sum scaling factor. If set to zero the variable scaling factor method is used
        osd_method="osd_e", #the OSD method. Choose from:  1) "osd_e", "osd_cs", "osd0"
        osd_order=0 #the osd search depth
        )   
        
        print(f"Error Rate: {p_error}")
        logicalErrorCount=0
        for i in range(100):
            error = localRandom.choice([0,1], size=code.N, replace=True, p=[1 - p_error, p_error])
            phyErrors[k]+=np.sum(error)
            syndrome=A1.hx@error %2
            bpd.decode(syndrome)
            residual_error = (bpd.osdw_decoding+error) % 2
            #Decoding is successful if the residual error commutes with the logical operators
            if (A1.lx@residual_error%2).any():
                logicalErrorCount+=1
        print(f"Logical Errors: {logicalErrorCount}\n")
        berDictionary[f"ber_{numberOfIterations[iters]}_iterations"][k]=logicalErrorCount
        k +=1
    marker = marker +1
    


markers = ['-', '-o', '-^', '-+', '-*', '-x', '-s', '-d']
fig1, ax1 = plt.subplots(1, 1, layout='constrained')
ax1.set(title='A1 Code: BP+OSD Decoding Performance', xlabel='Physical Error Rate', ylabel='Rate of logical errors.')

for iters in range(len(numberOfIterations)):
    ax1.loglog(totalRange, berDictionary[f"ber_{numberOfIterations[iters]}_iterations"]/numberOfShots, markers[iters], label = f"BP+OSD Decoding for A1.hz with {numberOfIterations[iters]} BP iterations.")    
    #ax1.loglog(totalRange, phyErrors/(numberOfShots * code.N), '-', label = 'Physical Errors, actual (sanity check).')
# #ax1.axvspan(0, 0.1, facecolor='lightblue', alpha=0.3, 'Region tested by IBM')   # Left side background
# #ax1.axvspan(0.1, 0.5, facecolor='lightgreen', alpha=0.3, 'Region by Panteleev and Kalachev') # Right side background
ax1.legend()
ax1.grid()
ax1.grid(which="minor", color="0.9")
